In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import glob
import json
import csv
from PIL import Image
import random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision.models import resnet18, resnet152
from torchsummary import summary

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm


Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Yolanda_Task2

In [ ]:
def set_seed(seed):
    """
    Use this to set ALL the random seeds to a fixed value and take out any randomness from cuda kernels
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = True  ##uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. -
    torch.backends.cudnn.enabled   = True

    return True

device = 'cpu'
if torch.cuda.device_count() > 0 and torch.cuda.is_available():
    print("Cuda installed! Running on GPU!")
    device = 'cuda'
else:
    print("No GPU available!")

In [ ]:
data_dir = '/content/drive/MyDrive/Selected_Storms_curated'
csvpath = '/content/drive/MyDrive/storm_df.csv'

In [ ]:
# do not use timeseries data
from dataloader_norm import StormDatasetN
test_size = 0.3
trainset = StormDatasetN(data_dir,csvpath, train=True,storm='full', test_size=test_size)
testset = StormDatasetN(data_dir,csvpath, train=False,storm='full', test_size=test_size)
print(len(trainset))
print(len(testset))

In [ ]:
# use timeseries data
from dataloader_norm import WindowStormDatasetN
test_size = 0.1
trainset = WindowStormDatasetN(data_dir,csvpath, train=True,storm='full', test_size=test_size)
testset = WindowStormDatasetN(data_dir,csvpath, train=False,storm='full', test_size=test_size)
print(len(trainset))
print(len(testset))

## Architechture##



### Simple CNN with 3 convolution layers##

We first used the simplest cnn model to test the structure (only used images to predict the wind speed).

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, outdim):
        super(SimpleCNN, self).__init__()

        inputsize = 366

        # Define the convolutional layers
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.2),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.2),
        )

        self.fc = nn.Sequential(
            nn.Linear(256 * (inputsize // 8) * (inputsize // 8), outdim),
            nn.ReLU(),
            )

    def forward(self, x):
        x = self.conv(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        return x

### CNN model in reference

We used the CNN model we found online and tried to imporve the results.

In [ ]:
class CNNRe(nn.Module):
    def __init__(self):
        super(CNNRe, self).__init__()
        self.model_ft = models.resnet18(pretrained=True)
        num_ftrs = self.model_ft.fc.in_features
        # Define the first fully connected layer.
        self.fc1 = nn.Sequential(nn.Linear(num_ftrs, 64), nn.ReLU())
        # Define the second fully connected layer followed
        #by a ReLU activation and the final linear layer for regression.
        self.fc2 = nn.Sequential(nn.Linear(64, 64), nn.ReLU(), nn.Linear(64, 1))
        self.dp = nn.Dropout(0.2)

    def forward(self, x):
        x = self.model_ft.conv1(x)
        x = self.model_ft.bn1(x)
        x = self.model_ft.relu(x)
        x = self.model_ft.maxpool(x)

        x = self.model_ft.layer1(x)
        x = self.model_ft.layer2(x)
        x = self.model_ft.layer3(x)
        x = self.model_ft.layer4(x)

        x = self.model_ft.avgpool(x).squeeze(-1).squeeze(-1)
        x = self.dp(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

### Resnet

Tried to train different resnet as well
(such as resnet18, resnet50, resnet152).

In [ ]:
class Resnet(nn.Module):
    def __init__(self, outdim):
        super(Resnet, self).__init__()
        self.resnet18 = resnet18(pretrained=True)
        self.resnet18.fc = nn.Sequential(
            nn.Linear(self.resnet18.fc.in_features, outdim),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.resnet18(x)
        return x

In [ ]:
class CNN_resnet(nn.Module):
    '''
    This model used a trained ResNet152.
    '''
    def __init__(self):
        super(CNN_resnet, self).__init__()
        self.cnn = resnet152(pretrained=True)  # out_features=1000
        self.fc = nn.Sequential(
            nn.Linear(1000, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64,1),
        )

    def forward(self, x, x_ft):
        x = self.cnn(x)
        x = self.fc(x)
        return x.squeeze()

### CNN+LSTM model##

During this process, we also tried to use CNN+LSTM model to train and predict wind speed for all features.

We tried to modify the CNN model to find the most appropriate architecture(utilizing the CNN models described above).



In [ ]:
class CNNLSTM(nn.Module):
    def __init__(self, indim=256, timedim=3):
        super(CNNLSTM, self).__init__()
        # # resnet
        # self.cnn = Resnet(indim)
        # indim += timedim
        # self.lstm = nn.LSTM(input_size=indim, hidden_size=256, num_layers=3, batch_first=True)

        # simpleCNN
        self.cnn = SimpleCNN(indim)
        indim += timedim
        self.lstm = nn.LSTM(input_size=indim, hidden_size=256, num_layers=3, batch_first=True)

        self.fc = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64,1),
        )

    def forward(self, x, x_ft):
        xsh = x.shape
        if len(xsh)==5:  # (bs, seq_len, C, H, W)
            x = x.reshape(-1, *xsh[2:])
            x = self.cnn(x)
            x = x.reshape(xsh[0], xsh[1], -1)
            x = torch.cat((x, x_ft), dim=-1)  # add features
            out, hidden = self.lstm(x)
            x = self.fc(out[:, -1, :])
            return x.squeeze()
        else:
            x = self.cnn(x)
            x = torch.cat((x, x_ft), dim=1)  # add features
            out, hidden = self.lstm(x.unsqueeze(0))
            x = self.fc(out[:, -1, :])
            return x.squeeze()

### Problem with LSTM model
![loss](figs/resnet-lstm-loss.png)

![predict plot](figs/resnet-lstm-res.png)

It is obviously to see what the problem is: we got all the same predictions when using this model.

We attempted the following approaches; however, it is evident that the issue is unresolved. Clearly, the issue did not lie here:

* change activate function (relu/leaky-relu)
* reduce learning rate (0.05/0.01/0.001/0.0001)
* add dropout and batchnorm
* use or not use relative_time and other features
* change normalize (normalize images to [0,1] or [-1,1], norm or not norm relative_time)
* change model architecture from resnet18 to a more simple one (3-layer convolution) or a more complex one (resnet50)
* add more linear layer after LSTM

## Here is results using above models.

It can be seen that while some of the RMSE values produce promising results, the R-squared values are not satisfactory. Moreover, these results were obtained without solving the previously mentioned LSTM problem.


| model | rmse | r2 |
| ------- | ------- | ------- |
| SimpleCNN + LSTM | 22.028 | -0.137 |
| resnet18 + LSTM | 20.724 | -0.006 |
| resnet50 + LSTM | 5.920 | -0.919 |
| SimpleCNN | 7.749 | -2.276 |
| resnet152 | 11.670 | 0.763 |
| CNN in ref| 62.1424|       |

in which the best model is resnet152:

![loss](figs/resnet-loss.png)

![predict plot](figs/resnet-res.png)

## training loop

Here are the train and evaluate models we used to produce the loss images and predict results.

In [ ]:
def train(model, optimizer, criterion, data_loader):
    '''
    Function to train the model.
    '''
    model.train()
    train_loss, train_rmse = 0, 0
    pbar = tqdm(data_loader)
    for data in pbar:
        X = data['image'].to(device)
        time = data['relative_time'].to(device)
        time_diff = data['time_diff'].to(device)
        ocean = data['ocean'].to(device)
        x_ft = torch.cat([time.unsqueeze(dim=-1),time_diff.unsqueeze(dim=-1), ocean.unsqueeze(dim=-1)], dim = -1).float()
        y = data['wind_speed'].float().to(device)

        optimizer.zero_grad()
        y_pred = model(X, x_ft)
        loss = criterion(y_pred, y)
        loss.backward()
        train_loss += loss*X.size(0)
        train_rmse += torch.sqrt(loss)*X.size(0)
        optimizer.step()
        pbar.set_description(f"loss: {loss:.4f}")

    train_loss = train_loss/len(data_loader.dataset)
    train_rmse = train_rmse/len(data_loader.dataset)
    return train_loss, train_rmse # Return the average loss and RMSE

def evaluate(model, data_loader):
    '''
    Function to evaluate the model
    '''
    model.eval()
    y_true, y_pred = [], []
    for data in data_loader:
        with torch.no_grad():
            X = data['image'].to(device)
            time = data['relative_time'].to(device)
            time_diff = data['time_diff'].to(device)
            ocean = data['ocean'].to(device)
            x_ft = torch.cat([time.unsqueeze(dim=-1),time_diff.unsqueeze(dim=-1), ocean.unsqueeze(dim=-1)], dim = -1).float()
            y = data['wind_speed'].to(device)
            # predict the wind speeds
            y_p = model(X, x_ft)
            y_true.append(y.cpu().numpy())
            y_pred.append(y_p.cpu().numpy())
    # Concat true and predicted wind speeds.
    y_pred = np.concatenate(y_pred, 0)
    y_pred = np.round(y_pred)
    y_true = np.concatenate(y_true, 0)
    # Calculate MSE, RMSE and R-squared.
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    print('rmse: ', rmse)
    print('r2: ', r2)
    return y_true, y_pred # Return true and predicted wind speeds.

In [ ]:
def train_model(model,trainset, n_epoch=20, batchsize=16, lr=0.001, weight_decay=1e-6):
    set_seed(42)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[i*10 for i in range(n_epoch//10)], gamma=0.5)

    # load data for training data
    trainloader = DataLoader(trainset, batchsize, shuffle=True, num_workers=4)

    # model save path
    savepath = os.path.join('checkpoints','cnn')
    if os.path.exists(savepath)==False:
        os.mkdir(savepath)

    start_epoch = 1
    modellist = sorted(os.listdir(savepath))
    if len(modellist)>0:
        checkpoint = modellist[-1]
        model.load_state_dict(torch.load(os.path.join(savepath, checkpoint)))
        start_epoch += int(checkpoint.split('.')[0])

    # train the model
    loss = []
    rmse = []

    for epoch in range(start_epoch, n_epoch+start_epoch):
        train_loss, train_rmse = train(model, optimizer, criterion, trainloader)
        loss.append(train_loss.item())
        rmse.append(train_rmse.item())
        if scheduler:
            scheduler.step()

        # pbar.set_description(f"loss: {train_loss.item():.4f}, mse: {train_mse.item():.4f}")

        # save checkpoints
        if (np.mod(epoch, 5) == 0):
            torch.save(model.state_dict(), os.path.join(savepath, "{:03d}.pth".format(epoch)))

    # plot loss and accuracy on trainset
    fig, ax = plt.subplots(1,2, figsize=(8,4))
    ax[0].plot(list(range(n_epoch)), loss, label='loss')
    ax[1].plot(list(range(n_epoch)), rmse, c='orange', label='mse')
    ax[0].set_xlabel('epoch')
    ax[0].set_title('loss')
    ax[1].set_xlabel('epoch')
    ax[1].set_title('rmse')
    plt.show()

def evaluate_model(model, testset, batchsize=64):
    testloader = DataLoader(testset, batchsize, shuffle=False, num_workers=4)
    y_true, y_pred = evaluate(model, testloader)
    print(y_pred)
    x = np.arange(len(y_true))
    df = pd.DataFrame({'true': y_true, 'pred': y_pred})
    # df = df.sort_values(by='true')
    plt.plot(x, df['true'], color='blue', label='true')
    plt.plot(x, df['pred'], color='red', label='predict')
    plt.legend(loc='best')
    plt.show()
